# LIBRARIES

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# DATASET

In [2]:

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)



26422272it [00:49, 530177.95it/s]                              


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



29696it [00:00, 201593.68it/s]                          


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



4422656it [00:11, 386595.28it/s]                             


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



6144it [00:00, ?it/s]                   

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw

Processing...
Done!



c:\Users\markg\venv\torch-venv\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# NEURAL NET

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)

# HYPERPARAMETERS

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# LOSS FUNCTION

In [5]:

loss_fn = nn.CrossEntropyLoss()

# OPTIMIZER

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# TRAINING

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306147  [    0/60000]
loss: 2.302284  [ 6400/60000]
loss: 2.290239  [12800/60000]
loss: 2.286166  [19200/60000]
loss: 2.296227  [25600/60000]
loss: 2.283005  [32000/60000]
loss: 2.273202  [38400/60000]
loss: 2.268317  [44800/60000]
loss: 2.264401  [51200/60000]
loss: 2.269440  [57600/60000]
Test Error: 
 Accuracy: 33.3%, Avg loss: 2.251235 

Epoch 2
-------------------------------
loss: 2.247412  [    0/60000]
loss: 2.257284  [ 6400/60000]
loss: 2.218485  [12800/60000]
loss: 2.223961  [19200/60000]
loss: 2.251416  [25600/60000]
loss: 2.235266  [32000/60000]
loss: 2.215973  [38400/60000]
loss: 2.203359  [44800/60000]
loss: 2.197989  [51200/60000]
loss: 2.220419  [57600/60000]
Test Error: 
 Accuracy: 32.8%, Avg loss: 2.178470 

Epoch 3
-------------------------------
loss: 2.177101  [    0/60000]
loss: 2.195122  [ 6400/60000]
loss: 2.117975  [12800/60000]
loss: 2.126375  [19200/60000]
loss: 2.187031  [25600/60000]
loss: 2.168182  [32000/600